In [41]:
import pandas as pd
from neo4j import GraphDatabase

pd.set_option('display.max_columns',None)

df = pd.read_csv('reviews_withsentiment.csv')

In [42]:
df.head(100)

,Unnamed: 0,ReviewTitle,RestaurantAddress,RestaurantName,Review,Food,RestaurantReview,FoodReview,RestaurantFeature(spacy),FoodFeature(spacy),RestaurantFeature(vader),FoodFeature(vader)
0,0,"""Life is a combination of magic and noodles.""","11 Lorong 3 Toa Payoh, Singapore",Chef Kang’s Noodle House,Featuring Wonton Noodles from Chef Kang's Nood...,NaN,NaN,featuring wonton noodles from chef kang's nood...,NaN,NaN,NaN,NaN
1,1,#SoNotHungry 🤷🏻‍♀️ .,"215R Upper Thomson Road, Singapore",One Man Coffee (Upper Thomson),@igsg #igsg #singapore #foodpornasia @burpple ...,coffee,,NaN,NaN,NaN,NaN,NaN
2,2,#Throwback to dinner at the new @mimirestauran...,"3A River Valley Road, Singapore",Mimi,"Interesting ambience, great service, and delec...",NaN,"interesting ambience, great service, and delec...","very glad we got to try this, will definitely ...","great,delectable,modern,chinese","glad,got,try,definitely return,😋,yang,stout,in...","great,delectable","glad,definitely,stout"
3,3,#breakfast #OOiaJ #overnightoatsinajar #eatcle...,Singapore,-,-,NaN,-,NaN,NaN,NaN,NaN,NaN
4,4,"#breakfast grapes, strawberries, banana, Pure ...",Singapore,-,-,NaN,-,NaN,NaN,NaN,NaN,NaN
5,5,#burpple,"112 East Coast Road, Singapore",GastroSmiths,-,NaN,-,NaN,NaN,NaN,NaN,NaN
6,6,#burpple #beefbowl,"112 East Coast Road, Singapore",GastroSmiths,-,NaN,-,NaN,NaN,NaN,NaN,NaN
7,7,"#burpple Apple Split (think banana split), but...","112 East Coast Road, Singapore",GastroSmiths,-,NaN,-,NaN,NaN,NaN,NaN,NaN
8,8,#coldbrewsaturdays for hot weather with @boots...,Singapore,Singapore,The coffee is acceptably strong as well!,coffee,NaN,the coffee is acceptably strong as well!,NaN,acceptably strong,NaN,"acceptably,strong"
9,9,#dimsum with family 😊 #FotoRus #roastedpork #p...,"391 Orchard Road, Singapore",Crystal Jade Palace Restaurant (Ngee Ann City),-,NaN,-,NaN,NaN,NaN,NaN,NaN


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081 entries, 0 to 1080
Data columns (total 12 columns):
Unnamed: 0                  1081 non-null int64
ReviewTitle                 1081 non-null object
RestaurantAddress           1063 non-null object
RestaurantName              1081 non-null object
Review                      1081 non-null object
Food                        759 non-null object
RestaurantReview            916 non-null object
FoodReview                  707 non-null object
RestaurantFeature(spacy)    651 non-null object
FoodFeature(spacy)          690 non-null object
RestaurantFeature(vader)    518 non-null object
FoodFeature(vader)          546 non-null object
dtypes: int64(1), object(11)
memory usage: 101.4+ KB


In [54]:
from py2neo import Graph, Node, Relationship
# Input the user name and password set in the neo4j graph
graph = Graph("bolt://localhost:7687", auth=("neo4j","1234"))
graph.delete_all()

In [55]:
for index, row in df.iterrows():
    # Create nodes from data set
    review = Node('Review', title = row['ReviewTitle'], features = row['RestaurantFeature(spacy)'])
    restaurant = Node('Restaurant', name = row['RestaurantName'], address = row['RestaurantAddress'])
    food = Node('Food', name = row['Food'], features = row['FoodFeature(spacy)'])
    
    # Prevent duplicated nodes
    graph.merge(review,'Review','title')
    graph.merge(restaurant,'Restaurant','name')
    graph.merge(food,'Food','name')
    
    # Create unique relationships
    review_about_restaurant = Relationship(review, "About", restaurant)
    graph.merge(review_about_restaurant)
    
    review_contains_food = Relationship(review, "Contains", food)
    graph.merge(review_contains_food)
    
    food_servedby_restaurant = Relationship(food, "Servedby", restaurant)
    graph.merge(food_servedby_restaurant)

In [52]:
graph.run("MATCH (n) RETURN n").to_table()

n
"(_2280:Review {features: '\U0001f924,hosting', title: 'Cakes'})"
"(_2281:Food {features: 'cakechicago,cheesecakebelgian,dark,layered,good,tasty,served,warm,nice,overly cheesy,goes', name: 'banana,banana cake,bean,beans,cake,cakes,cheese,cheesecake,chocolate,coffee,nut'})"
"(_2282:Review {features: 'free,bad,thought,available,regular,sized,hosting', title: 'California Sunrise / Strawberry Cheesecake / Choc Cookie Crumble'})"
"(_2283:Food {features: 'orange,turned,refreshing,strawberry,light,flavour,sweet,favourite,goes,choc,uses,dark,rich,diluted,chocolate,mention,hot,earl,grey,definitely generous,create,thick,fragrant,accompanied,cheesecake', name: 'bean,beans,berry,cake,cheese,cheesecake,chocolate,coffee,cookie crumble,dark chocolate,earl,milk,strawberry,tea,vanilla'})"
"(_2284:Review {features: '238251)operating', title: 'Calling all Unagi Lovers.'})"
"(_2285:Restaurant {address: '30 Robertson Quay, Singapore', name: 'Unagiya Ichinoji (Robertson Quay)'})"
"(_2286:Food {features: 'unagi,visit,\U0001f4cd,unagiya,quay', name: 'bento,pho,rice,unagi'})"
"(_2287:Review {features: 'missed,getting,messy,satisfying,lacked,good,tasted,substantial', title: 'Came looking for Burgs to find the space occupied with a new halal burger stall.'})"
"(_2288:Restaurant {address: '505 Beach Road, Singapore', name: 'Ashes Burnnit! (Golden Mile Food Centre)'})"
"(_2289:Food {features: 'offered,similar,truffle,proper,decent,thick,cut', name: 'burger,cheese,cheese burger,fries,mushroom,thick cut fries,truffle'})"
